In [1]:
# general imports
from spacy.language import Language
from spacy.tokens import Token, Doc

# static sentiment imports
from nltk.corpus import opinion_lexicon

# contextual sentiment imports
import numpy as np
import torch 
from transformers_interpret import SequenceClassificationExplainer
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
)

import unicodedata
import difflib

def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if unicodedata.category(c) != 'Mn')


@Language.factory("contextual_sentiment", 
                  default_config={
                    "model_name": "distilbert-base-uncased-finetuned-sst-2-english",
                    "dataset": None,
                    "device": "cuda"
                })
def create_contextual_sentiment_component(nlp: Language,
                                          name: str,
                                          model_name: str = None,
                                          dataset: str = None, 
                                          device: str = None):
    if not Token.has_extension("contextual_sentiment"):
        Token.set_extension("contextual_sentiment", default = 'unknown')
    return ContextualSentimentComponent(model_name=model_name,
                                        dataset=dataset,
                                        device=device)

class ContextualSentimentComponent:
    def __init__(self, 
                 model_name=None,
                 dataset=None, 
                 device='cuda'):
      
        self.dataset = dataset
        self.model_name = model_name
        self.device = 'cuda' if torch.cuda.is_available() and 'cuda' in device else 'cpu'

        if self.model_name:
            self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name).to(self.device)
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        else:
            # find huggingface model to provide rationalized output
            from huggingface_hub import HfApi

            api = HfApi()
            modelIds = api.list_models(filter=("pytorch", "dataset:" + dataset, "sibyl"))
            if modelIds:
                modelId = getattr(modelIds[0], 'modelId')
                print('Using ' + modelId + ' to rationalize keyphrase selections.')
                self.model = AutoModelForSequenceClassification.from_pretrained(modelId).to(self.device)
                self.tokenizer = AutoTokenizer.from_pretrained(modelId)
            else: 
                raise "No model found. Please provide one via `model_name`."
        self.interpreter = SequenceClassificationExplainer(self.model, self.tokenizer)

    def __call__(self, doc: Doc):   

        # get sentiment attributions
        attributions = self.interpreter(text=doc.text)[1:-1]
        doc_is_negative = 1 if not self.interpreter.predicted_class_index else 0
        tokens, weights = zip(*attributions)
        #print(tokens)
        words, weights = merge_bpe(tokens, weights)

        weights = align_tokens([strip_accents(t.text).lower() for t in doc], words, weights.tolist())
        weights *= -1 if doc_is_negative else 1
            
        assert len(doc) == len(weights)
        
        # add attributions to doc
        for sp_token, weight in zip(doc, weights):
            if sp_token.is_stop:
                sp_token._.contextual_sentiment = "exclude"
            else:
                sp_token._.contextual_sentiment = stringify_sentiment(weight)

        return doc

def merge_bpe(tok, boe, chars="##"):
    len_chars = len(chars)

    new_tok = []
    new_boe = []

    emb = []
    append = ""
    for t, e in zip(tok[::-1], boe[::-1]):
        t += append
        emb.append(e)
        if t.startswith(chars):
            append = t[len_chars:]
        else:
            append = ""
            new_tok.append(t)
            new_boe.append(np.stack(emb).mean(axis=0))
            emb = []  
    new_tok = np.array(new_tok)[::-1]
    new_boe = np.array(new_boe)[::-1]
    return new_tok, new_boe


def align_tokens(doc_tok, tok, boe):

    new_embs = []
    
    print(doc_tok)
    print(tok)
    print(boe)
    
    seq = difflib.SequenceMatcher(None, doc_tok, tok)
    edits = seq.get_opcodes()
    
    for i, (op, from_start, from_end, to_start, to_end) in enumerate(edits):
        print(op, doc_tok[from_start:from_end], tok[to_start:to_end])
        if op == 'equal':
            new_embs += boe[to_start:to_end]
        elif op == 'insert':
            cur_embs = boe[to_start:to_end]

            end = from_end 
            start = end
            
            k = max(i - 1, 0)

            while k >= 0:
                if edits[k][0] != 'equal':
                    start = edits[k][1]
                    break
                k -= 1
            cur_embs += new_embs[start:end]
            avg_emb = np.stack(cur_embs).mean(axis=0)
            new_embs[start:end] = [avg_emb] * (end - start)
            
            print('insert:', f"[{start},{end}]", avg_emb)
            print(new_embs)
            
        elif op == 'replace':
            avg_emb = np.stack(boe[to_start:to_end]).mean(axis=0)
            print(avg_emb)
            for j in range(from_start, from_end):
                new_embs.append(avg_emb)
        elif op == 'delete':
            for j in range(from_start,from_end):
                print(0)
                new_embs.append(0)
                
    print()
    print(len(new_embs), new_embs)
    print(len(doc_tok), doc_tok)

    return np.array(new_embs)


def stringify_sentiment(weight):
    ranges = {
        (-1.0, -0.33): "neg",
        (-0.33, 0.33): "neutral",
        (0.33,  1.0 ): "pos"
    }
    for (lower_bound, upper_bound), value in ranges.items():
        if lower_bound <= weight < upper_bound:
            return value
    return "NA"

2022-08-30 20:19:31.204392: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import spacy

model = spacy.load("en_core_web_sm")
model.add_pipe("contextual_sentiment")

In [3]:
model("the u.s.")

['the', 'u.s', '.']
['the' 'u' '.' 's' '.']
[0.10717678812159016, 0.26897030445194425, 0.2625891017642, 0.919751340989627, -0.035672759823451916]
equal ['the'] ['the']
replace ['u.s'] ['u']
0.26897030445194425
equal ['.'] ['.']
insert [] ['s' '.']
insert: [1,3] 0.3539094968455798
[0.10717678812159016, 0.3539094968455798, 0.3539094968455798]

3 [0.10717678812159016, 0.3539094968455798, 0.3539094968455798]
3 ['the', 'u.s', '.']


the u.s.

In [4]:
model('compete the original and in some event even turn it  a motion on your boundary ')

['compete', 'the', 'original', 'and', 'in', 'some', 'event', 'even', 'turn', 'it', ' ', 'a', 'motion', 'on', 'your', 'boundary']
['compete' 'the' 'original' 'and' 'in' 'some' 'event' 'even' 'turn' 'it'
 'a' 'motion' 'on' 'your' 'boundary']
[0.34632817856631876, -0.14151047081118742, -0.027213395481218008, -0.038571426975226565, 0.186820126424306, -0.04742325575178569, 0.14302118504600467, -0.24727113301817955, -0.058329286021442286, -0.1523758920555082, 0.2739872693677103, -0.15530674350104892, 0.1636893800870778, 0.5683726412074418, 0.5132278823519683]
equal ['compete', 'the', 'original', 'and', 'in', 'some', 'event', 'even', 'turn', 'it'] ['compete' 'the' 'original' 'and' 'in' 'some' 'event' 'even' 'turn' 'it']
delete [' '] []
0
equal ['a', 'motion', 'on', 'your', 'boundary'] ['a' 'motion' 'on' 'your' 'boundary']

16 [0.34632817856631876, -0.14151047081118742, -0.027213395481218008, -0.038571426975226565, 0.186820126424306, -0.04742325575178569, 0.14302118504600467, -0.24727113301817

compete the original and in some event even turn it  a motion on your boundary 

In [5]:
model("hide new secretions from the parental units  https://www.dictionary.com/browse/deteriorate ⛹🏼‍♀")

['hide', 'new', 'secretions', 'from', 'the', 'parental', 'units', ' ', 'https://www.dictionary.com/browse/deteriorate', '⛹', '🏼\u200d', '♀']
['hide' 'new' 'secretions' 'from' 'the' 'parental' 'units' 'https' ':' '/'
 '/' 'www' '.' 'dictionary' '.' 'com' '/' 'browse' '/' 'deteriorate'
 '[UNK]']
[0.24703614598585658, 0.163733745780639, 0.1963703796449008, 0.16658135609316782, 0.025846419132600777, 0.2100994137224555, 0.06613921698318256, -0.15449450251981292, -0.01990700585130143, 0.21180839390780776, 0.0849861038502238, 0.022299068388093837, 0.07901607027208031, 0.04375479629216791, 0.1557738573393934, 0.36043682690011, 0.15552628816638642, 0.06777157695539936, 0.27656359629951066, 0.3448403075655393, -0.1334578617662118]
equal ['hide', 'new', 'secretions', 'from', 'the', 'parental', 'units'] ['hide' 'new' 'secretions' 'from' 'the' 'parental' 'units']
replace [' ', 'https://www.dictionary.com/browse/deteriorate', '⛹', '🏼\u200d', '♀'] ['https' ':' '/' '/' 'www' '.' 'dictionary' '.' 'com'

hide new secretions from the parental units  https://www.dictionary.com/browse/deteriorate ⛹🏼‍♀

In [6]:
model("Wicked kickflip my dude!")

['wicked', 'kickflip', 'my', 'dude', '!']
['wicked' 'kickflip' 'my' 'dude' '!']
[0.7493978941924485, 0.3075497792453416, 0.21145802823003124, 0.12754079255844816, -0.03106573446496777]
equal ['wicked', 'kickflip', 'my', 'dude', '!'] ['wicked' 'kickflip' 'my' 'dude' '!']

5 [0.7493978941924485, 0.3075497792453416, 0.21145802823003124, 0.12754079255844816, -0.03106573446496777]
5 ['wicked', 'kickflip', 'my', 'dude', '!']


Wicked kickflip my dude!

## DF loading

In [5]:
import pandas as pd
import os.path as osp

In [6]:
result_pth = './results_a2t_sst2'

In [7]:
log_df = pd.read_csv(osp.join(result_pth,'log.csv'))
out_df = log_df[['original_text', 'perturbed_text', 'result_type']]
out_df

,original_text,perturbed_text,result_type
0,it 's a charming and often affecting journey .,it 's a charming and normally affecting journe...,Failed
1,unflinchingly bleak and desperate,unflinchingly grim and desperate,Failed
2,allows us to hope that nolan is poised to emba...,authorizes us to hopes that nolan is prepped t...,Failed
3,"the acting , costumes , music , cinematography...","the acting , costumes , music , cinematography...",Successful
4,"it 's slow -- very , very slow .","it 's slow -- very , very slower .",Failed
...,...,...,...
867,has all the depth of a wading pool .,has all the depths of a wading pool .,Successful
868,a movie with a real anarchic flair .,a films with a true anarchic flair .,Failed
869,a subject like this should inspire reaction in...,a subject like this should inspiring reply in ...,Successful
870,... is an arthritic attempt at directing by ca...,... is an arthritic endeavour at directing by ...,Successful


In [8]:
log_df

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,it 's a charming and often affecting journey .,it 's a charming and normally affecting journe...,0.000234,0.000557,1.0,1.0,1.0,14.0,Failed
1,unflinchingly bleak and desperate,unflinchingly grim and desperate,0.014329,0.154515,0.0,0.0,0.0,5.0,Failed
2,allows us to hope that nolan is poised to emba...,authorizes us to hopes that nolan is prepped t...,0.000426,0.283926,1.0,1.0,1.0,32.0,Failed
3,"the acting , costumes , music , cinematography...","the acting , costumes , music , cinematography...",0.003972,0.622039,1.0,0.0,1.0,8.0,Successful
4,"it 's slow -- very , very slow .","it 's slow -- very , very slower .",0.002048,0.002501,0.0,0.0,0.0,4.0,Failed
...,...,...,...,...,...,...,...,...,...
867,has all the depth of a wading pool .,has all the depths of a wading pool .,0.185487,0.851692,0.0,1.0,0.0,2.0,Successful
868,a movie with a real anarchic flair .,a films with a true anarchic flair .,0.000322,0.000818,1.0,1.0,1.0,5.0,Failed
869,a subject like this should inspire reaction in...,a subject like this should inspiring reply in ...,0.136275,0.518427,0.0,1.0,0.0,13.0,Successful
870,... is an arthritic attempt at directing by ca...,... is an arthritic endeavour at directing by ...,0.003134,0.734410,0.0,1.0,0.0,11.0,Successful


In [9]:
transformation_log = pd.read_csv('../results/transformation.csv', index_col=0, names = ["transformation_id","transformation",
            "prev_text", "after_text", "prev_target", "after_target","from_modified_indices", "to_modified_indices", "changes"])
transformation_log

,transformation,prev_text,after_text,prev_target,after_target,from_modified_indices,to_modified_indices,changes
transformation_id,,,,,,,,
0,"{""module_name"": ""textattack.transformations.wo...",0,1,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
1,"{""module_name"": ""textattack.transformations.wo...",2,3,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
2,"{""module_name"": ""textattack.transformations.wo...",2,4,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
3,"{""module_name"": ""textattack.transformations.wo...",2,5,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
4,"{""module_name"": ""textattack.transformations.wo...",2,6,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
...,...,...,...,...,...,...,...,...
2115,"{""module_name"": ""textattack.transformations.wo...",1233,2135,-1,-1,{2},{2},"['replace: [2,3]-[2,3]']"
2116,"{""module_name"": ""textattack.transformations.wo...",1233,2136,-1,-1,{2},{2},"['replace: [2,3]-[2,3]']"
2117,"{""module_name"": ""textattack.transformations.wo...",1233,2137,-1,-1,{2},{2},"['replace: [2,3]-[2,3]']"


In [10]:
edge_to_transformation = transformation_log.set_index(['prev_text', 'after_text'])
edge_to_transformation

transformation  \
prev_text after_text                                                      
0         1           {"module_name": "textattack.transformations.wo...   
2         3           {"module_name": "textattack.transformations.wo...   
          4           {"module_name": "textattack.transformations.wo...   
          5           {"module_name": "textattack.transformations.wo...   
          6           {"module_name": "textattack.transformations.wo...   
...                                                                 ...   
1233      2135        {"module_name": "textattack.transformations.wo...   
          2136        {"module_name": "textattack.transformations.wo...   
          2137        {"module_name": "textattack.transformations.wo...   
          2138        {"module_name": "textattack.transformations.wo...   
          2139        {"module_name": "textattack.transformations.wo...   

                      prev_target  after_target from_modified_indices  \
prev_text after_text                                                    
0         1                    -1            -1                   {3}   
2         3                    -1            -1                   {3}   
          4                    -1            -1                   {3}   
          5                    -1            -1                   {3}   
          6                    -1            -1                   {3}   
...                           ...           ...                   ...   
1233      2135                 -1            -1                   {2}   
          2136                 -1            -1                   {2}   
          2137                 -1            -1                   {2}   
          2138                 -1            -1                   {2}   
          2139                 -1            -1                   {2}   

                     to_modified_indices                   changes  
prev_text after_text                                                
0         1                          {3}  ['replace: [3,4]-[3,4]']  
2         3                          {3}  ['replace: [3,4]-[3,4]']  
          4                          {3}  ['replace: [3,4]-[3,4]']  
          5                          {3}  ['replace: [3,4]-[3,4]']  
          6                          {3}  ['replace: [3,4]-[3,4]']  
...                                  ...                       ...  
1233      2135                       {2}  ['replace: [2,3]-[2,3]']  
          2136                       {2}  ['replace: [2,3]-[2,3]']  
          2137                       {2}  ['replace: [2,3]-[2,3]']  
          2138                       {2}  ['replace: [2,3]-[2,3]']  
          2139                       {2}  ['replace: [2,3]-[2,3]']  

[2120 rows x 6 columns]

In [11]:
edges_df = transformation_log[['prev_text','after_text']]
forward_edges_df = edges_df.set_index('prev_text')
backward_edges_df = edges_df.set_index('after_text')

In [12]:
id_to_text = pd.read_csv('../results/text.csv', index_col="text_id", names = ["text_id", "text"])
text_to_id = pd.read_csv('../results/text.csv', index_col="text", names = ["text_id", "text"])

## Transformation History

In [13]:
from lineage import InferQuery
from collections import defaultdict
from textattack.shared.utils import words_from_text
queryAPI = InferQuery(dir_pth='./results_a2t_sst2')

2022-07-25 20:38:05.960456: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [14]:
api_trace = queryAPI.get_trace_of_output_idx(25)
api_trace

[(2504,
  "nicks , seemingly uncertain what 's going to make people laugh , runs the gamut from stale parody to raunchy sex gags to formula romantic comedy . ",
  0.0),
 ("{'class': 'WordSwapEmbedding', 'max_candidates': 20, 'embedding': WordEmbedding}",
  '{21}',
  '{21}'),
 [('replace', (21, 22), (21, 22))],
 (2505,
  "nicks , seemingly uncertain what 's going to make people laugh , runs the gamut from stale parody to raunchy sex gags to formulas romantic comedy . ",
  0.0),
 ("{'class': 'WordSwapEmbedding', 'max_candidates': 20, 'embedding': WordEmbedding}",
  '{14}',
  '{14}'),
 [('replace', (14, 15), (14, 15))],
 (2529,
  "nicks , seemingly uncertain what 's going to make people laugh , runs the gamut from archaic parody to raunchy sex gags to formulas romantic comedy . ",
  0.0),
 ("{'class': 'WordSwapEmbedding', 'max_candidates': 20, 'embedding': WordEmbedding}",
  '{12}',
  '{12}'),
 [('replace', (12, 13), (12, 13))],
 (2545,
  "nicks , seemingly uncertain what 's going to make

In [15]:
all_traces = queryAPI.get_traces_of_all_outputs()

In [16]:
all_traces[0]

[(2, "it 's a charming and often affecting journey . ", 1.0),
 ("{'class': 'WordSwapEmbedding', 'max_candidates': 20, 'embedding': WordEmbedding}",
  '{5}',
  '{5}'),
 [('replace', (5, 6), (5, 6))],
 (28, "it 's a charming and normally affecting journey . ", 1.0)]

In [17]:
len(all_traces)

806

In [18]:
import spacy
sp = spacy.load('en_core_web_sm')

In [19]:
def to_sentiment_category(word):
    if  word in stop_words:
        return 'exclude'
    elif word in pos_list:
        return 'pos'
    elif word in neg_list:
        return 'neg'
    else:
        return 'neutral'

In [20]:
from nltk.corpus import opinion_lexicon
from nltk.corpus import stopwords
import nltk
nltk.download('opinion_lexicon','stopwords')

pos_list=set(opinion_lexicon.positive())
neg_list=set(opinion_lexicon.negative())
stop_words = set(set(stopwords.words('english')))

[nltk_data] Downloading package opinion_lexicon to stopwords...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [55]:
class FeatureText:
    def __init__(text):
        self.text = text
        self.features_dict = {}
        
    def add_features(name, feature_extraction_func):
        self.features_dict[name] = feature_extraction_func(self.text)
        return self.features_dict
    
    def get_features(name):
        return self.features_dict[name]

In [59]:
class EditsSummary:
    def __init__(all_texts: FeatureText):
        self.all_texts = all_texts
        self.edits ={}
        
    def get_edits(edit_id, ):
        self.edits
        

In [60]:
all_edits = {}
all_texts = {}
for record in all_traces:
    final_label = record[-1][-1]
    
    for i,step in enumerate(record):
        if isinstance(step, list):
            for edit in step:
                op = edit[0]
                from_span = edit[1]
                to_span = edit[2]
                from_text = tuple(words_from_text(record[i-2][1])[from_span[0]:from_span[1]])
                to_text = tuple(words_from_text(record[i+1][1])[to_span[0]:to_span[1]])
                
                from_text_id = record[i-2][0]
                to_text_id = record[i+1][0]
                all_texts[record[i-2][0]] = record[i-2][1]
                all_texts[record[i+1][0]] = record[i+1][1]
                
                from_label = record[i-2][2]
                to_label = record[i+1][2]
                
                
                final_text_id = record[-1][0]
                
                # to category
                from_cat = to_sentiment_category(from_text[0])
                to_cat = to_sentiment_category(to_text[0])
                
                result_type = record[i+1][2]
                if op not in all_edits:
                    all_edits[op] = defaultdict(dict)
                if to_cat not in all_edits[op][from_cat]:
                    all_edits[op][from_cat][to_cat]={}
                if (from_label, final_label) not in all_edits[op][from_cat][to_cat]:
                    all_edits[op][from_cat][to_cat][(from_label, final_label)] = []
                    
                all_edits[op][from_cat][to_cat][(from_label, final_label)].append((from_text, to_text, from_text_id, to_text_id, final_text_id, final_label)) 

In [61]:
all_edits

{'replace': defaultdict(dict,
             {'neutral': {'neutral': {(1.0,
                 1.0): [(('often',),
                  ('normally',),
                  2,
                  28,
                  28,
                  1.0), (('hope',), ('hopes',), 128, 146, 260, 1.0), (('allows',),
                  ('authorizes',),
                  146,
                  172,
                  260,
                  1.0), (('embark',),
                  ('engage',),
                  172,
                  260,
                  260,
                  1.0), (('refreshingly',), ('cheerfully',), 346, 349, 529, 1.0), (('film',),
                  ('films',),
                  390,
                  450,
                  529,
                  1.0), (('women',),
                  ('female',),
                  450,
                  529,
                  529,
                  1.0), (('blend',), ('amalgam',), 693, 718, 774, 1.0), (('romance',),
                  ('romanticism',),
             

In [62]:
edit_freqs = defaultdict(list)
for op in all_edits.keys():
    for from_cat in all_edits[op].keys():
        for to_cat in all_edits[op][from_cat].keys():
            for label_pair in all_edits[op][from_cat][to_cat].keys():
                edit_freqs[label_pair].append((len(all_edits[op][from_cat][to_cat][label_pair]), op, from_cat, to_cat))

In [63]:
for label_pair in edit_freqs.keys():
    print(label_pair)
    edit_freqs[label_pair] = sorted(edit_freqs[label_pair], reverse=True)
    for edit in edit_freqs[label_pair]:
        print(f"\t{edit}")

(1.0, 1.0)
	(601, 'replace', 'neutral', 'neutral')
	(114, 'replace', 'pos', 'neutral')
	(102, 'replace', 'pos', 'pos')
	(42, 'replace', 'neg', 'neg')
	(37, 'replace', 'neg', 'neutral')
	(28, 'replace', 'neutral', 'neg')
	(27, 'replace', 'neutral', 'pos')
	(17, 'replace', 'pos', 'neg')
	(5, 'replace', 'neutral', 'exclude')
	(1, 'replace', 'neg', 'pos')
(1.0, 0.0)
	(110, 'replace', 'neutral', 'neutral')
	(29, 'replace', 'neg', 'neg')
	(28, 'replace', 'pos', 'neutral')
	(14, 'replace', 'pos', 'pos')
	(9, 'replace', 'neutral', 'neg')
	(8, 'replace', 'neg', 'neutral')
	(4, 'replace', 'neutral', 'exclude')
	(2, 'replace', 'pos', 'neg')
	(2, 'replace', 'neutral', 'pos')
	(1, 'replace', 'neg', 'pos')
(0.0, 0.0)
	(527, 'replace', 'neutral', 'neutral')
	(112, 'replace', 'neg', 'neg')
	(54, 'replace', 'neg', 'neutral')
	(41, 'replace', 'pos', 'pos')
	(34, 'replace', 'neutral', 'pos')
	(30, 'replace', 'pos', 'neutral')
	(16, 'replace', 'neutral', 'neg')
	(11, 'replace', 'neg', 'pos')
	(9, 'replace

In [65]:
label_percentage = {}
for label_pair in edit_freqs.keys():
    freq_sum = sum([tup[0] for tup in edit_freqs[label_pair]])
    edits_dict = {}
    for edit in edit_freqs[label_pair]:
        edits_dict[edit[1:]] = edit[0]
    label_percentage[label_pair] = edits_dict
        
for label_pair in label_percentage.keys():
    if label_pair[0] != label_pair[1]:
        print(label_pair)
        original_pair = (label_pair[0], label_pair[0])
        
        total = sum(label_percentage[label_pair].values()) + \
            sum(label_percentage[original_pair].values())
        for edit in sorted(label_percentage[label_pair].keys()):
            cur_percent = label_percentage[label_pair][edit]
            if edit in label_percentage[original_pair]:
                original_percent = label_percentage[original_pair][edit]
                print(f"\t{edit}, {cur_percent}/{original_percent + cur_percent} = {cur_percent / (original_percent + cur_percent) * 100:.2f}%")

(1.0, 0.0)
	('replace', 'neg', 'neg'), 29/71 = 40.85%
	('replace', 'neg', 'neutral'), 8/45 = 17.78%
	('replace', 'neg', 'pos'), 1/2 = 50.00%
	('replace', 'neutral', 'exclude'), 4/9 = 44.44%
	('replace', 'neutral', 'neg'), 9/37 = 24.32%
	('replace', 'neutral', 'neutral'), 110/711 = 15.47%
	('replace', 'neutral', 'pos'), 2/29 = 6.90%
	('replace', 'pos', 'neg'), 2/19 = 10.53%
	('replace', 'pos', 'neutral'), 28/142 = 19.72%
	('replace', 'pos', 'pos'), 14/116 = 12.07%
(0.0, 1.0)
	('replace', 'neg', 'neg'), 19/131 = 14.50%
	('replace', 'neg', 'neutral'), 13/67 = 19.40%
	('replace', 'neg', 'pos'), 4/15 = 26.67%
	('replace', 'neutral', 'exclude'), 1/10 = 10.00%
	('replace', 'neutral', 'neg'), 4/20 = 20.00%
	('replace', 'neutral', 'neutral'), 114/641 = 17.78%
	('replace', 'neutral', 'pos'), 9/43 = 20.93%
	('replace', 'pos', 'neutral'), 6/36 = 16.67%
	('replace', 'pos', 'pos'), 15/56 = 26.79%


In [68]:
class bcolors:
    PURPLE = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [69]:
import torch
import numpy as np
import difflib

In [70]:
import transformers

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2")
model.eval()
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2", use_fast=True)

In [71]:
reversed_texts = []
final_texts = []
original_results = []
for edit in all_edits['replace']['neg']['neg'][(0.0, 0.0)]:
    before_text = all_texts[edit[2]]
    left = before_text.find(edit[0][0])
    right = left + len(edit[0][0])
    
    print(before_text[:left] + bcolors.GREEN + before_text[left:right] +bcolors.ENDC + before_text[right:])

    after_text = all_texts[edit[3]]
    left = after_text.find(edit[1][0])
    right = left + len(edit[1][0])
    
    print(after_text[:left] + bcolors.RED + after_text[left:right] +bcolors.ENDC + after_text[right:])
    print()

    final_text = all_texts[edit[4]]
    final_label = edit[5]
    reverted_text = final_text.replace(edit[1][0], edit[0][0])
    final_texts.append(final_text)
    original_results.append(final_label)
    reversed_texts.append(reverted_text)

unflinchingly bleak and desperate 
unflinchingly grim and desperate 

it 's slow -- very , very slow . 
it 's slow -- very , very slower . 

even horror fans will most likely not find what they 're seeking with trouble every day ; the movie lacks both thrills and humor . 
even horror fans will most likely not find what they 're seeking with trouble every day ; the movie lacked both thrills and humor . 

even horror fans will most likely not find what they 're seeking with trouble every day ; the movie lacked both thrills and comedy . 
even horror fans will most likely not find what they 're seeking with difficulty every day ; the movie lacked both thrills and comedy . 

the action switches between past and present , but the material link is too tenuous to anchor the emotional connections that purport to span a 125-year divide . 
the action switches between past and present , but the material link is too fragile to anchor the emotional connections that purport to span a 125-year divide 

- text data size reduction
    - text data collection - human evaluation/connotation --> correction of semantics/grammar
    - tagging words - (sentiment labels, POS, constituent component) 
    - 

In [72]:
new_results = []
for text in reversed_texts:
    encoding = tokenizer.encode_plus(
      text,
      max_length=160,
      add_special_tokens=True, # Add '[CLS]' and '[SEP]'
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',  # Return PyTorch tensors
    )
    output = model(
      input_ids=encoding['input_ids'],
      attention_mask=encoding['attention_mask']
    )
    new_results.append(torch.argmax(output.logits, dim=1).item())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/coraline/anaconda3/envs/dpml/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [73]:
new_results = np.array(new_results)

In [74]:
comp_result_count = defaultdict(list)
for i, (old, new) in enumerate(zip(original_results, new_results)):
    comp_result_count[(int(old), new)].append(i)

for k in comp_result_count.keys():
    print(k, len(comp_result_count[k]))

(0, 0) 112


In [75]:
for i in comp_result_count[(1, 0)]:
    parsed_a = words_from_text(reversed_texts[i])
    parsed_b = words_from_text(final_texts[i])
    seq = difflib.SequenceMatcher(None, parsed_a, parsed_b)
    changes = seq.get_opcodes()
    
    for (tag, i1, i2, j1, j2) in changes:
        if tag == 'equal' and (j2 - j1) == (i2 - i1):
            continue
        elif tag == 'replace' or tag == 'insert':
            parsed_a[i1] = bcolors.GREEN + parsed_a[i1]
            if i2 == len(parsed_a):
                parsed_a.append(bcolors.ENDC)
            else:
                parsed_a[i2] = bcolors.ENDC + parsed_a[i2]
            
            parsed_b[j1] = bcolors.RED + parsed_b[j1]
            
            if j2 == len(parsed_b):
                parsed_b.append(bcolors.ENDC)
            else:
                parsed_b[j2] = bcolors.ENDC + parsed_b[j2]
        elif tag == 'delete':
            parsed_a[i1] = bcolors.GREEN + parsed_a[i1]
            parsed_a[i2] = bcolors.ENDC + parsed_a[i2]

    print('revert:')
    print(' '.join(parsed_a))
    print('perturbed:')
    print(' '.join(parsed_b))
    print()

## POS tags

In [32]:
import spacy
sp = spacy.load('en_core_web_sm')

In [33]:
from lineage import InferQuery
from collections import defaultdict
from textattack.shared.utils import words_from_text
queryAPI = InferQuery(dir_pth='./results_a2t_sst2')

In [34]:
all_traces = queryAPI.get_traces_of_all_outputs()

In [35]:
def to_pos_tag(sent):
    sent_doc=sp(sent)
    pos_tags = []
    for i in range(len(sent_doc)):
        pos_tags.append(sent_doc[i].pos_)
    return pos_tags

In [36]:
to_pos_tag('I love cats')

['PRON', 'VERB', 'NOUN']

In [43]:
all_edits = {}
all_texts = {}
for record in all_traces:
    final_label = record[-1][-1]
    
    for i,step in enumerate(record):
        if isinstance(step, list):
            for edit in step:
                op = edit[0]
                from_span = edit[1]
                to_span = edit[2]
                from_text = tuple(words_from_text(record[i-2][1])[from_span[0]:from_span[1]])
                to_text = tuple(words_from_text(record[i+1][1])[to_span[0]:to_span[1]])
                
                from_text_id = record[i-2][0]
                to_text_id = record[i+1][0]
                all_texts[record[i-2][0]] = record[i-2][1]
                all_texts[record[i+1][0]] = record[i+1][1]
                
                from_pos_list = to_pos_tag(' '.join(words_from_text(record[i-2][1])))
                to_pos_list = to_pos_tag(' '.join(words_from_text(record[i+1][1])))
                
                from_label = record[i-2][2]
                to_label = record[i+1][2]
                
                final_text_id = record[-1][0]
                
                # to category
                from_cat = from_pos_list[from_span[0]]
                to_cat = to_pos_list[to_span[0]]
                
                result_type = record[i+1][2]
                if op not in all_edits:
                    all_edits[op] = defaultdict(dict)
                if to_cat not in all_edits[op][from_cat]:
                    all_edits[op][from_cat][to_cat]={}
                if (from_label, to_label) not in all_edits[op][from_cat][to_cat]:
                    all_edits[op][from_cat][to_cat][(from_label, to_label)] = []
                    
                all_edits[op][from_cat][to_cat][(from_label, to_label)].append((from_text, to_text, from_text_id, to_text_id, final_text_id, final_label)) 

In [45]:
edit_freqs = defaultdict(list)
for op in all_edits.keys():
    for from_cat in all_edits[op].keys():
        for to_cat in all_edits[op][from_cat].keys():
            for label_pair in all_edits[op][from_cat][to_cat].keys():
                edit_freqs[label_pair].append((len(all_edits[op][from_cat][to_cat][label_pair]), op, from_cat, to_cat))

In [46]:
for label_pair in edit_freqs.keys():
    print(label_pair)
    edit_freqs[label_pair] = sorted(edit_freqs[label_pair], reverse=True)
    for edit in edit_freqs[label_pair]:
        print(f"\t{edit}")

(1.0, 1.0)
	(367, 'replace', 'NOUN', 'NOUN')
	(200, 'replace', 'ADJ', 'ADJ')
	(197, 'replace', 'VERB', 'VERB')
	(96, 'replace', 'ADV', 'ADV')
	(28, 'replace', 'ADJ', 'NOUN')
	(20, 'replace', 'PUNCT', 'PUNCT')
	(19, 'replace', 'ADP', 'ADP')
	(18, 'replace', 'ADJ', 'VERB')
	(14, 'replace', 'VERB', 'NOUN')
	(14, 'replace', 'PRON', 'PRON')
	(13, 'replace', 'PROPN', 'PROPN')
	(11, 'replace', 'NOUN', 'VERB')
	(10, 'replace', 'DET', 'DET')
	(9, 'replace', 'CCONJ', 'CCONJ')
	(8, 'replace', 'NOUN', 'PROPN')
	(7, 'replace', 'VERB', 'ADJ')
	(7, 'replace', 'AUX', 'AUX')
	(6, 'replace', 'NOUN', 'ADJ')
	(5, 'replace', 'SCONJ', 'SCONJ')
	(5, 'replace', 'PROPN', 'NOUN')
	(5, 'replace', 'ADV', 'ADJ')
	(5, 'replace', 'ADJ', 'ADV')
	(4, 'replace', 'ADJ', 'PROPN')
	(2, 'replace', 'PRON', 'NOUN')
	(2, 'replace', 'PART', 'PART')
	(2, 'replace', 'ADJ', 'INTJ')
	(1, 'replace', 'SCONJ', 'ADP')
	(1, 'replace', 'PROPN', 'VERB')
	(1, 'replace', 'PRON', 'DET')
	(1, 'replace', 'DET', 'ADJ')
	(1, 'replace', 'ADV', '

In [49]:
label_percentage = {}
for label_pair in edit_freqs.keys():
    freq_sum = sum([tup[0] for tup in edit_freqs[label_pair]])
    edits_dict = {}
    for edit in edit_freqs[label_pair]:
        edits_dict[edit[1:]] = edit[0] 
    label_percentage[label_pair] = edits_dict
        
for label_pair in label_percentage.keys():
    if label_pair[0] != label_pair[1]:
        print(label_pair)
        original_pair = (label_pair[0], label_pair[0])
        
        total = sum(label_percentage[label_pair].values()) + \
            sum(label_percentage[original_pair].values())
        for edit in sorted(label_percentage[label_pair].keys()):
            cur_percent = label_percentage[label_pair][edit]
            if edit in label_percentage[original_pair]:
                original_percent = label_percentage[original_pair][edit]
                print(f"\t{edit}, {cur_percent}/{original_percent + cur_percent} = {cur_percent / (original_percent + cur_percent) * 100:.2f}%")

(0.0, 1.0)
	('replace', 'ADJ', 'ADJ'), 23/205 = 11.22%
	('replace', 'ADJ', 'NOUN'), 1/12 = 8.33%
	('replace', 'ADJ', 'VERB'), 3/13 = 23.08%
	('replace', 'ADP', 'ADP'), 1/15 = 6.67%
	('replace', 'ADV', 'ADJ'), 1/2 = 50.00%
	('replace', 'ADV', 'ADV'), 9/88 = 10.23%
	('replace', 'AUX', 'AUX'), 1/14 = 7.14%
	('replace', 'CCONJ', 'CCONJ'), 1/4 = 25.00%
	('replace', 'DET', 'DET'), 1/16 = 6.25%
	('replace', 'NOUN', 'ADJ'), 2/9 = 22.22%
	('replace', 'NOUN', 'NOUN'), 24/316 = 7.59%
	('replace', 'NOUN', 'PROPN'), 1/11 = 9.09%
	('replace', 'PRON', 'PRON'), 3/19 = 15.79%
	('replace', 'PUNCT', 'PUNCT'), 3/21 = 14.29%
	('replace', 'VERB', 'VERB'), 18/202 = 8.91%
(1.0, 0.0)
	('replace', 'ADJ', 'ADJ'), 16/216 = 7.41%
	('replace', 'ADJ', 'INTJ'), 2/4 = 50.00%
	('replace', 'ADJ', 'NOUN'), 2/30 = 6.67%
	('replace', 'ADJ', 'VERB'), 1/19 = 5.26%
	('replace', 'ADP', 'ADP'), 1/20 = 5.00%
	('replace', 'ADV', 'ADJ'), 1/6 = 16.67%
	('replace', 'ADV', 'ADV'), 8/104 = 7.69%
	('replace', 'AUX', 'AUX'), 2/9 = 22.22

In [52]:
reversed_texts = []
final_texts = []
original_results = []
for edit in all_edits['replace']['NOUN']['NOUN'][(0.0, 0.0)]:
    before_text = all_texts[edit[2]]
    left = before_text.find(edit[0][0])
    right = left + len(edit[0][0])
    
    print(before_text[:left] + bcolors.GREEN + before_text[left:right] +bcolors.ENDC + before_text[right:])

    after_text = all_texts[edit[3]]
    left = after_text.find(edit[1][0])
    right = left + len(edit[1][0])
    
    print(after_text[:left] + bcolors.RED + after_text[left:right] +bcolors.ENDC + after_text[right:])
    print()

    final_text = all_texts[edit[4]]
    final_label = edit[5]
    reverted_text = final_text.replace(edit[1][0], edit[0][0])
    final_texts.append(final_text)
    original_results.append(final_label)
    reversed_texts.append(reverted_text)

a occasionally tedious film . 
a occasionally tedious movies . 

or doing last year 's taxes with your ex-wife . 
or doing last year 's taxation with your ex-wife . 

it takes a inquisitive kind of laziness to squandering the talents of robert forster , anne meara , eugene levy , and reginald veljohnson all in the same movie . 
it takes a inquisitive kind of laziness to squandering the talents of robert forster , anne meara , eugene levy , and reginald veljohnson all in the same film . 

it takes a inquisitive kind of laziness to squandering the talents of robert forster , anne meara , eugene levy , and reginald veljohnson all in the same film . 
it takes a inquisitive genus of laziness to squandering the talents of robert forster , anne meara , eugene levy , and reginald veljohnson all in the same film . 

... the film undergo from a lack of humor ( something needed to balance out the violence ) ... 
... the film undergo from a lack of mood ( something needed to balance out the violen

In [ ]:
interjection